In [1]:
!pip install langchain
!pip install langchain-groq


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = "api_key",
    model_name = "llama3-70b-8192"
)

response = llm.invoke("The first person to walk on moon was...")
print(response.content)

That's an easy one! The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he took his first steps on the moon.


In [6]:
!pip install chromadb

In [27]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name = "new_collection")

In [28]:
collection.add(
    documents = [
        'This is about New York .',
        'This is another Delhi',
    ],
    ids = ['id1', 'id2'],
)

In [29]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This is about New York .', 'This is another Delhi'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [31]:
doc = collection.get(ids = ['id1'])
doc

{'ids': ['id1'],
 'embeddings': None,
 'documents': ['This is about New York .'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [44]:
collection.query(
    query_texts = ['Query is about big apple'],
    n_results = 2
)

{'ids': [['id1', 'id2']],
 'embeddings': None,
 'documents': [['This is about New York .', 'This is another Delhi']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.598836898803711, 1.8085007667541504]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [ ]:
# client.delete_collection(name="my_collection") 
# client.delete_collection(name="new_collection")

In [45]:
collection.delete(ids = all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [46]:
collection.add(
    documents = [
        "This is about New York.",
        "This i about Delhi."
    ],
    ids  = ['id1', 'id2'],
    metadatas = [
        {"url" : "https://en.wikipedia.org/wiki/New_York_City"},
        {"url" : "https://en.wikipedia.org/wiki/New_Delhi"}
    ]
)

In [47]:
results = collection.query(
    query_texts = ["Query is about Chhole Bhature"],
    n_results=2
)
results

{'ids': [['id2', 'id1']],
 'embeddings': None,
 'documents': [['This i about Delhi.', 'This is about New York.']],
 'uris': None,
 'data': None,
 'metadatas': [[{'url': 'https://en.wikipedia.org/wiki/New_Delhi'},
   {'url': 'https://en.wikipedia.org/wiki/New_York_City'}]],
 'distances': [[1.669273853302002, 1.9922130107879639]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [49]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)






















Nike Careers








































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Langua

In [50]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input = {'page_data' : page_data})
print(res.content)


Here is the extracted job postings in JSON format:

```
[
  {
    "role": "(Workplace) Lead, GPS",
    "experience": "",
    "skills": "",
    "description": "Job Location: London, London, United Kingdom Business Operations"
  },
  {
    "role": "6 Athlète Nike (Vendeur/euse en magasin) – H / F CDI 28h",
    "experience": "",
    "skills": "",
    "description": "Job Location: Claye, Seine-et-Marne, France Retail Stores"
  },
  {
    "role": "APLA, Supply Chain Sustainability, Senior",
    "experience": "",
    "skills": "",
    "description": "Job Location: Ciudad de México, Ciudad de Mexico, Mexico Corporate Services"
  },
  {
    "role": "Abteilungsleiter/in (BOH Coach) - 37.5H - NFS Wertheim (m/w/d)",
    "experience": "",
    "skills": "",
    "description": "Job Location: Wertheim, Baden-Wurttemberg, Germany Retail Stores"
  },
  {
    "role": "Abteilungsleiter/in (Coach) - 100% - NFS Landquart",
    "experience": "",
    "skills": "",
    "description": "Job Location: Landquart,

In [51]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': '(Workplace) Lead, GPS',
  'experience': '',
  'skills': '',
  'description': 'Job Location: London, London, United Kingdom Business Operations'},
 {'role': '6 Athlète Nike (Vendeur/euse en magasin) – H / F CDI 28h',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Claye, Seine-et-Marne, France Retail Stores'},
 {'role': 'APLA, Supply Chain Sustainability, Senior',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Ciudad de México, Ciudad de Mexico, Mexico Corporate Services'},
 {'role': 'Abteilungsleiter/in (BOH Coach) - 37.5H - NFS Wertheim (m/w/d)',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Wertheim, Baden-Wurttemberg, Germany Retail Stores'},
 {'role': 'Abteilungsleiter/in (Coach) - 100% - NFS Landquart',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Landquart, Graubunden, Switzerland Retail Stores'},
 {'role': 'Accounting Admin - Aventura',
  'experience': '',
  'skills': '',
  'descript

In [52]:
type(json_res)

list

In [54]:
!pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [56]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [58]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name = 'portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents = row['Techstack'],
            metadatas = {"links" : row['Links']},
            ids = [str([uuid.uuid4()])]
        )

In [59]:
links = collection.query(
    query_texts = ["Experience in Python", "Expertise in React"],
    n_results = 2,
)
links

{'ids': [["[UUID('91fb0bfa-c591-45f2-959a-897a2e692325')]",
   "[UUID('6300549c-af58-4b8e-9774-14c6b7fa118d')]"],
  ["[UUID('1f13f9b4-4a15-4056-a2e6-337c2f86564f')]",
   "[UUID('7588b050-b1cc-484d-9cde-a386448f9e62')]"]],
 'embeddings': None,
 'documents': [['Machine Learning, Python, TensorFlow',
   'Python, Django, MySQL'],
  ['React, Node.js, MongoDB', 'React Native, Node.js, MongoDB']],
 'uris': None,
 'data': None,
 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/python-portfolio'}],
  [{'links': 'https://example.com/react-portfolio'},
   {'links': 'https://example.com/react-native-portfolio'}]],
 'distances': [[0.9968848007555503, 1.0578701721493369],
  [1.1309200916978361, 1.2047648656063519]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [67]:
job

[{'role': '(Workplace) Lead, GPS',
  'experience': '',
  'skills': '',
  'description': 'Job Location: London, London, United Kingdom Business Operations'},
 {'role': '6 Athlète Nike (Vendeur/euse en magasin) – H / F CDI 28h',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Claye, Seine-et-Marne, France Retail Stores'},
 {'role': 'APLA, Supply Chain Sustainability, Senior',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Ciudad de México, Ciudad de Mexico, Mexico Corporate Services'},
 {'role': 'Abteilungsleiter/in (BOH Coach) - 37.5H - NFS Wertheim (m/w/d)',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Wertheim, Baden-Wurttemberg, Germany Retail Stores'},
 {'role': 'Abteilungsleiter/in (Coach) - 100% - NFS Landquart',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Landquart, Graubunden, Switzerland Retail Stores'},
 {'role': 'Accounting Admin - Aventura',
  'experience': '',
  'skills': '',
  'descript

In [68]:

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res_email = chain_email.invoke({"job_description" : str(job), "link_list" : links})
print(res_email.content)

Here is the cold email:

Subject: Streamline Your Operations with AtliQ's Automation Expertise

Dear Hiring Manager,

I came across your job posting for [Role] and was impressed by the scope of responsibilities and the emphasis on process optimization. As a business development executive at AtliQ, I believe our expertise in automation and software consulting can help your organization achieve its goals.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has successfully implemented automated tools that have transformed the way businesses operate.

I'd like to highlight a few examples from our portfolio that demonstrate our capabilities:

* We developed a machine learning-based solution using Python and TensorFlow for a leading retail company, resulting in a 30% reduction in operational costs. (https://example.com/ml-python-portfol